In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
import warnings
warnings.filterwarnings('ignore')

## Предобработка данных о погоде

In [283]:
# Чтение данных и удаление первой строки с единицами измерения
df = pd.read_csv('weather.csv', na_values="", parse_dates=['Datetime']).drop(index=0, axis=1)

# Названия колонок к PEP8
df.columns = df.columns.str.replace(' ', '_').str.lower()

# Приведение колонок к float, кроме колонки с датами
for col in df.columns[1:]:
    df[col] = df[col].astype(float)

#### Проверка на дубликаты

In [284]:
df.duplicated().sum(), df['datetime'].duplicated().sum()

(np.int64(0), np.int64(0))

#### Работа с пропусками

In [285]:
df.isna().sum() # Проверка наличия пропусков

datetime                0
temperature            55
precipitation_total    30
wind_gust               0
wind_speed             16
cloud_cover_total      20
sunshine_duration      12
dtype: int64

Так как мы работаем с временными рядами, в заполнении пропусков может помочь **интерполяция**, которая отталкивается от значений соседних точек.

##### ДО

In [286]:
df.iloc[1003:1015]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1004,2023-05-12 19:00:00,14.07,0.10,18.72,6.13,28.00,42.07
1005,2023-05-12 20:00:00,13.69,0.00,14.40,5.51,26.00,0.00
1006,2023-05-12 21:00:00,NaN,0.00,12.96,6.57,20.00,0.00
1007,2023-05-12 22:00:00,NaN,0.00,14.04,7.24,38.00,0.00
1008,2023-05-12 23:00:00,NaN,0.00,14.04,6.62,38.00,0.00
1009,2023-05-13 00:00:00,NaN,0.00,13.68,7.73,25.00,0.00
1010,2023-05-13 01:00:00,NaN,0.00,12.96,7.17,23.70,0.00
1011,2023-05-13 02:00:00,NaN,0.00,12.60,8.56,23.10,0.00
1012,2023-05-13 03:00:00,NaN,0.00,11.16,6.73,59.00,0.00
1013,2023-05-13 04:00:00,NaN,0.00,10.80,6.64,77.00,1.44


##### ПОСЛЕ

In [287]:
for col in df.columns[1:]:
    df[col] = df[col].interpolate()
    
df.iloc[1003:1015]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1004,2023-05-12 19:00:00,14.07,0.10,18.72,6.13,28.00,42.07
1005,2023-05-12 20:00:00,13.69,0.00,14.40,5.51,26.00,0.00
1006,2023-05-12 21:00:00,13.49,0.00,12.96,6.57,20.00,0.00
1007,2023-05-12 22:00:00,13.29,0.00,14.04,7.24,38.00,0.00
1008,2023-05-12 23:00:00,13.08,0.00,14.04,6.62,38.00,0.00
1009,2023-05-13 00:00:00,12.88,0.00,13.68,7.73,25.00,0.00
1010,2023-05-13 01:00:00,12.68,0.00,12.96,7.17,23.70,0.00
1011,2023-05-13 02:00:00,12.48,0.00,12.60,8.56,23.10,0.00
1012,2023-05-13 03:00:00,12.27,0.00,11.16,6.73,59.00,0.00
1013,2023-05-13 04:00:00,12.07,0.00,10.80,6.64,77.00,1.44


#### Работа со значениями

In [288]:
# Перевод порыва и скорости ветра в м/с и округление до 1 знака после запятой
df[['wind_gust', 'wind_speed']] = (df[['wind_gust', 'wind_speed']] / 3.6).round(1)

# Округление продолжительности солнечного сияния, температуры и облачности до целого
df[['sunshine_duration', 'temperature', 'cloud_cover_total']] = df[['sunshine_duration', 'temperature', 'cloud_cover_total']].round(0)

df.iloc[1234:1240]

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
1235,2023-05-22 10:00:00,23.00,0.00,4.60,0.90,8.00,55.00
1236,2023-05-22 11:00:00,24.00,0.00,5.10,1.10,4.00,58.00
1237,2023-05-22 12:00:00,25.00,0.00,5.00,1.40,4.00,58.00
1238,2023-05-22 13:00:00,25.00,0.10,5.00,1.90,9.00,55.00
1239,2023-05-22 14:00:00,26.00,0.60,6.00,1.00,18.00,49.00
1240,2023-05-22 15:00:00,25.00,0.50,6.40,1.10,19.00,48.00


#### Проверка на выбросы

In [290]:
df.describe()

,datetime,temperature,precipitation_total,wind_gust,wind_speed,cloud_cover_total,sunshine_duration
count,3672,3672.00,3672.00,3672.00,3672.00,3672.00,3672.00
mean,2023-06-16 11:30:00.000000256,18.09,0.10,5.80,2.04,40.98,21.54
min,2023-04-01 00:00:00,-2.00,0.00,0.80,0.00,0.00,0.00
25%,2023-05-09 05:45:00,13.00,0.00,3.70,1.20,15.00,0.00
50%,2023-06-16 11:30:00,18.00,0.00,5.10,1.70,32.00,15.00
75%,2023-07-24 17:15:00,23.00,0.00,7.30,2.60,67.00,43.00
max,2023-08-31 23:00:00,36.00,5.80,19.80,8.30,100.00,60.00
std,NaN,6.84,0.34,2.90,1.18,31.31,21.89


Все значения находятся в норме, выбросов не наблюдается

#### Выгрузка новой таблицы

In [291]:
new_units_of_measure = ["", "°C", "mm", "m/s", "m/s", "%", "min"] # Скорости теперь в м / с

first_row = pd.DataFrame([new_units_of_measure], columns=df.columns, index=[0]) # Добавление в начало таблицы

df = pd.concat([first_row, df], ignore_index=True)

df.to_csv('processed_weather.csv', index=False)

## Предобработка данных о поездках

In [269]:
# Чтение данных и удаление первой строки с единицами измерения
df = pd.read_csv('rides.csv', na_values="", parse_dates=['Start Date', 'End Date'])

# Названия колонок к PEP8
df.columns = df.columns.str.replace(' ', '_').str.lower()

# Приведение колонок к float, кроме колонки с датами
for col in df.columns[7:]:
    df[col] = df[col].astype(float)

### Проверка на дубликаты

Данные, представленные в таблице rides.csv могут дублироваться, поэтому убирать дубликаты не требуется

### Работа с пропусками

In [142]:
df.isna().sum() # Проверка наличия пропусков

id                   0
start_date           0
end_date           579
start_location       0
start_district       0
end_location         0
end_district         0
distance          1233
promo                0
dtype: int64

Наблюдаются пропуски с дате окончания поездки и дистанции поездки.

* Пропуски в end_date будем заполнять на основании пройденной дистанции

* Пропуски в distance будем заполнять на основании времени поездки

#### Работа с пропусками в end_date

Рассчитывать время конца поездки будем на основании медианы средней скорости во время пути и дистанции поездки.

Формула: Конец_поездки = начало_поездки + (дистанция / медиана_средней_скорости)

ДО

In [80]:
df.iloc[1015:1018]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo
1015,101302,2023-04-23 00:00:22,NaT,Трудовая,Центральный,Первомайская,Северо-Западный,4497.0,0.0
1016,101303,2023-04-23 00:03:19,2023-04-23 00:33:48,Маяковского,центральный,Пионерская,Заречный,4503.0,0.0
1017,101304,2023-04-23 00:05:09,2023-04-23 00:22:41,Железнодорожная,Центральный,Красная,Северо-Западный,2744.0,0.0


ПОСЛЕ

In [270]:
# Создаем колонну ride_duration, которая содержит длительность поездки в минутах
df['ride_duration'] = df.apply(lambda row: (row.end_date - row.start_date).seconds / 60, axis = 1)

# Создаем колонку average_speed, которая хранит среднюю скорость на протяжении всего пути в км/ч
df['average_speed'] = df.apply(lambda row: (row.distance / 1000) / (row.ride_duration / 60), axis=1)

median_average_speed = df['average_speed'].median()

# Заполнение пустых значений в столбце 'end_date'
df['end_date'] = df['end_date'].fillna(
    df['start_date'] + pd.to_timedelta((df['distance'] / 1000) / median_average_speed, unit='h')
)

df.iloc[1015:1018]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1015,101302,2023-04-23 00:00:22,2023-04-23 00:29:58.042904291,Трудовая,Центральный,Первомайская,Северо-Западный,4497.00,0.00,NaN,NaN
1016,101303,2023-04-23 00:03:19,2023-04-23 00:33:48.000000000,Маяковского,центральный,Пионерская,Заречный,4503.00,0.00,30.48,8.86
1017,101304,2023-04-23 00:05:09,2023-04-23 00:22:41.000000000,Железнодорожная,Центральный,Красная,Северо-Западный,2744.00,0.00,17.53,9.39


#### Работа с просками в distance

Рассчитывать дистанцию поездки будем на основании медианы средней скорости во время пути и длительности поездки.

Формула: Дистанция = медиана_средней_скорости * время_поездки

ДО

In [81]:
df.iloc[1037:1040]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo
1037,101322,2023-04-23 01:01:46,2023-04-23 01:47:24,Центральная,Юго-Восточный,Партизанская,Ленинский,NaN,0.0
1038,101323,2023-04-23 01:09:32,2023-04-23 01:23:44,сосновая,Юго-Восточный,юбилейная,юго-восточный,2025.0,0.0
1039,101324,2023-04-23 01:24:41,2023-04-23 01:43:10,ул.восточная,Октябрьский,ул.лермонтова,Ленинский,3124.0,0.0


ПОСЛЕ

In [271]:
# Заполнение пустых значений в столбце 'distance'
df['distance'] = df['distance'].fillna(median_average_speed * (df['ride_duration'] / 60) * 1000)

# Округление дистанции до целого
df['distance'] = df['distance'].round(0)

df.iloc[1037:1040]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1037,101322,2023-04-23 01:01:46,2023-04-23 01:47:24,Центральная,Юго-Восточный,Партизанская,Ленинский,6933.00,0.00,45.63,NaN
1038,101323,2023-04-23 01:09:32,2023-04-23 01:23:44,сосновая,Юго-Восточный,юбилейная,юго-восточный,2025.00,0.00,14.20,8.56
1039,101324,2023-04-23 01:24:41,2023-04-23 01:43:10,ул.восточная,Октябрьский,ул.лермонтова,Ленинский,3124.00,0.00,18.48,10.14


#### Проверка работы

In [192]:
df2 = df.drop(['ride_duration', 'average_speed'], axis=1) # Удаляем уже ненужные стоблцы

df2.isna().sum() # Проверка наличия пропусков

id                0
start_date        0
end_date          0
start_location    0
start_district    0
end_location      0
end_district      0
distance          0
promo             0
dtype: int64

### Проверка на выбросы

In [272]:
pd.set_option('display.float_format', '{:.2f}'.format)
df2.describe()

,id,start_date,end_date,distance,promo
count,102255.00,102255,102255,102255.00,102255.00
mean,149024.06,2023-06-13 21:03:26.564109568,2023-06-13 21:30:19.129662208,27010.23,0.02
min,100340.00,2023-04-22 09:01:03,2023-04-22 09:24:45,1.00,0.00
25%,124682.50,2023-05-20 21:41:13,2023-05-20 22:08:53.500000,3043.00,0.00
50%,149028.00,2023-06-14 15:22:15,2023-06-14 15:48:43,3702.00,0.00
75%,173372.50,2023-07-08 19:53:17.500000,2023-07-08 20:20:23.500000,4431.00,0.00
max,197711.00,2023-07-31 23:57:01,2023-08-14 07:25:03.057205720,7569789.00,1.00
std,28110.57,NaN,NaN,307856.12,0.14


В distance присутствуют очень сильные выбросы, вероятно, они вызваны ошибками в показаниях датчиков

Так как выбросы в distance аномально большие или маленькие, то самым простым способом будет вручную отфильтровать выбросы и исправить ошибочные значения, не прибегая к интерквартильному размаху или стандартным отклонениям

Выбросы будем исправиль по аналогии с пропусками.

ДО

In [275]:
df.iloc[1011:1013]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1011,101298,2023-04-22 23:52:20,2023-04-23 00:36:54,ул. строительная,северо-западный,ул.майская,Октябрьский,4659.00,0.00,44.57,6.27
1012,101299,2023-04-22 23:54:39,2023-04-23 00:23:58,ул. дачная,Центральный,ул.свободы,Ленинский,4273850.00,0.00,29.32,8746.94


ПОСЛЕ

In [276]:
df['distance'] = df['distance'].where(
    (df['distance'] >= 10) & (df['distance'] <= 50000), 
    median_average_speed * (df['ride_duration'] / 60) * 1000
)


df.iloc[1011:1013]

,id,start_date,end_date,start_location,start_district,end_location,end_district,distance,promo,ride_duration,average_speed
1011,101298,2023-04-22 23:52:20,2023-04-23 00:36:54,ул. строительная,северо-западный,ул.майская,Октябрьский,4659.00,0.00,44.57,6.27
1012,101299,2023-04-22 23:54:39,2023-04-23 00:23:58,ул. дачная,Центральный,ул.свободы,Ленинский,4453.85,0.00,29.32,8746.94


#### Проверка работы

In [277]:
df = df.drop(['ride_duration', 'average_speed'], axis=1) # Удаляем теперь ненужные колонны таблицы
df.describe()

,id,start_date,end_date,distance,promo
count,102255.00,102255,102255,102234.00,102255.00
mean,149024.06,2023-06-13 21:03:26.564109568,2023-06-13 21:30:19.129662208,3828.74,0.02
min,100340.00,2023-04-22 09:01:03,2023-04-22 09:24:45,519.00,0.00
25%,124682.50,2023-05-20 21:41:13,2023-05-20 22:08:53.500000,3101.00,0.00
50%,149028.00,2023-06-14 15:22:15,2023-06-14 15:48:43,3732.61,0.00
75%,173372.50,2023-07-08 19:53:17.500000,2023-07-08 20:20:23.500000,4448.59,0.00
max,197711.00,2023-07-31 23:57:01,2023-08-14 07:25:03.057205720,12675.36,1.00
std,28110.57,NaN,NaN,1050.17,0.14


### Работа со значениями

In [278]:
# Приводим значения направлений движения к единому формату
df['end_district'] = df['end_district'].str.lower().str.strip().str.replace(" ", '-')
df['start_district'] = df['start_district'].str.lower().str.strip().str.replace(" ", '-')

# Приводим значения начальных и конечных точек к единому формату
df['start_location'] = df['start_location'].str.lower().str.strip().str.replace(".", ' ').str.replace("  ", ' ')
df['start_location'] = df['start_location'].str.replace('ул ', '')

df['end_location'] = df['end_location'].str.lower().str.strip().str.replace(".", ' ').str.replace("  ", ' ')
df['end_location'] = df['end_location'].str.replace('ул ', '')

### Выгрузка новой таблицы

In [280]:
df.to_csv('processed_rides.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')

## Объединение таблиц

Для более удобного анализа данных стоит объединить имеющиеся таблицы

In [ ]:
rides_df = pd.read_csv('processed_rides.csv', parse_dates=['start_date', 'end_date'])
weather_df = pd.read_csv('processed_weather.csv', parse_dates=['datetime']).drop(index=0, axis=1)

rides_df['date'] = rides_df['start_date'].dt.date
weather_df['date'] = weather_df['datetime'].dt.date

rides_df['rounded_start_time'] = rides_df['start_date'].dt.round('h')


merge_df = pd.merge_asof(
    rides_df.sort_values('rounded_start_time'), 
    weather_df.sort_values('datetime'), 
    left_on='rounded_start_time', 
    right_on='datetime', 
    direction='backward'
)

merge_df = merge_df.drop(['date_x', 'date_y', 'rounded_start_time'], axis=1)

merge_df.to_csv('rides_&_weather.csv', index=False)